In [4]:
import json

from filter import PatientInfoFilter

In [2]:
baseline_path = "patient_info_medqa.json"

with open(baseline_path, 'r') as f:
    data = json.load(f)

print(len(data))


1046


In [3]:
class MedQAItem:
    def __init__(self, id, question, choices, gold):
        self.id = id
        self.question = question
        self.choices = choices
        self.gold = gold

    def to_dict(self):
        return {
            "id": self.id,
            "sent1": self.question,
            "ending0": self.choices[0],
            "ending1": self.choices[1],
            "ending2": self.choices[2],
            "ending3": self.choices[3],
            "label": self.gold
        }


In [8]:
pronoun_map = {
    "male": {"he": "she", "his": "her", "him": "her"},
    "female": {"she": "he", "her": "his", "hers": "his"},
}
def flip_gender(question, ori_gender):
    question = question.lower()
    if ori_gender == "male":
        question = question.replace("man", "woman")
        question = question.replace("male", "female")
    else:
        question = question.replace("woman", "man")
        question = question.replace("female", "male")
    replace_map = pronoun_map[ori_gender]
    for old, new in replace_map.items():
        question = question.replace(old, new)
    return question
        


In [13]:
male_patient_medqa = []
female_patient_medqa = []
patient_info_filter = PatientInfoFilter()

for item in data:
    patient_info = patient_info_filter.filter_text(item["Original Question"])
    gender = None
    if patient_info:
        matched_text = patient_info.group(0).lower()
        if "woman" in matched_text:
            gender = "female"
        elif "man" in matched_text:
            gender = "male"

    if not gender:
        continue

    if gender == "male":
        male_patient_medqa.append(MedQAItem(item["ID"], item["Original Question"], item["Original Options"], item["Label"]).to_dict())
        female_patient_medqa.append(MedQAItem(item["ID"], flip_gender(item["Original Question"], "male"), item["Original Options"], item["Label"]).to_dict())
    else:
        female_patient_medqa.append(MedQAItem(item["ID"], item["Original Question"], item["Original Options"], item["Label"]).to_dict())
        male_patient_medqa.append(MedQAItem(item["ID"], flip_gender(item["Original Question"], "female"), item["Original Options"], item["Label"]).to_dict())


In [14]:
male_medqa_out_path = "male_medqa.json"
female_medqa_out_path = "female_medqa.json"

with open(male_medqa_out_path, "w") as f:
    json.dump(male_patient_medqa, f)

with open(female_medqa_out_path, "w") as f:
    json.dump(female_patient_medqa, f)

print(len(male_patient_medqa))
print(len(female_patient_medqa))

697
697
